In [ ]:
import xml.etree.ElementTree as ET
import random
import os

def load_call_files(directory):
    calls_data = {}
    for filename in os.listdir(directory):
        if filename.endswith(".xml"):  # Only process XML files
            file_path = os.path.join(directory, filename)
            tree = ET.parse(file_path)
            root = tree.getroot()

            # Extract the call title and other details
            call_title = root.get("title")  # Assuming 'title' is an attribute of the root
            formations = []

            # Retrieve valid formations for each call
            for tam in root.findall("tam"):
                formation = tam.get("from")
                if formation:
                    formations.append(formation)

            # Store call data
            calls_data[call_title] = {
                "formations": formations,
                "file_path": file_path
            }
    return calls_data

def parse_formations(file_path):
    formations = {}
    tree = ET.parse(file_path)
    root = tree.getroot()
    for formation in root.findall("formation"):
        name = formation.get("name")
        dancers = []
        for dancer in formation.findall("dancer"):
            dancers.append({
                "gender": dancer.get("gender"),
                "x": float(dancer.get("x")),
                "y": float(dancer.get("y")),
                "angle": float(dancer.get("angle"))
            })
        formations[name] = dancers
    return formations

def parse_moves(file_path):
    moves = {}
    tree = ET.parse(file_path)
    root = tree.getroot()
    for path in root.findall("path"):
        name = path.get("name")
        movements = []
        for movement in path.findall("movement"):
            movements.append({
                "hands": movement.get("hands"),
                "beats": float(movement.get("beats")),
                "cx1": float(movement.get("cx1")),
                "cy1": float(movement.get("cy1")),
                "cx2": float(movement.get("cx2")),
                "cy2": float(movement.get("cy2")),
                "x2": float(movement.get("x2")),
                "y2": float(movement.get("y2"))
            })
        moves[name] = movements
    return moves


class DancePatternGenerator:
    def __init__(self, formations, moves, calls_directory):
        self.formations = formations
        self.moves = moves
        self.calls_data = load_call_files(calls_directory)

    def generate_sequence(self, start_formation, max_calls=10):
        sequence = []
        current_formation = start_formation

        for _ in range(max_calls):
            # Find valid calls for the current formation
            valid_calls = [call_title for call_title, call_data in self.calls_data.items()
                           if self.is_valid_transition(call_data, current_formation)]

            if not valid_calls:
                break
            
            selected_call_title = random.choice(valid_calls)
            sequence.append(selected_call_title)
            
            # Apply moves and update formation
            current_formation = self.update_formation(selected_call_title, current_formation)
            print(f"Selected call: {selected_call_title}, New formation: {current_formation}")

        return sequence

    def is_valid_transition(self, call_data, current_formation):
        if current_formation in call_data["formations"]:
            return True
        return False

    def update_formation(self, selected_call_title, current_formation):
        """
        Updates the formation after a call is made. If the formation cannot be determined,
        it returns the current formation.
        """
        # Retrieve the moves associated with the selected call (if any)
        moves_for_call = self.moves.get(selected_call_title, [])
        dancers = self.formations.get(current_formation, [])

        for move in moves_for_call:
            for dancer in dancers:
                dancer["x"] += move.get("x2", 0)
                dancer["y"] += move.get("y2", 0)

        # Use the current formation for now until more complex logic is developed
        updated_formation = current_formation

        return updated_formation



# Paths to data files
formations = parse_formations("../assets/src/formations.xml")
moves = parse_moves("../assets/src/moves.xml")

# Initialize generator with loaded data
generator = DancePatternGenerator(formations, moves, "../assets/b1")

# Generate a sequence with random calls
generated_sequence = generator.generate_sequence("Ocean Waves", max_calls=5)
print("Generated Dance Sequence:")
print(generated_sequence)


Selected call: Step Thru, New formation: Columns
Selected call: Step Thru, New formation: Columns
Selected call: Step Thru, New formation: Columns
Selected call: Step Thru, New formation: Columns
Selected call: Step Thru, New formation: Columns
Generated Dance Sequence:
['Step Thru', 'Step Thru', 'Step Thru', 'Step Thru', 'Step Thru']


Patter Generator V1 Includes:

1. Adapt the First Call Based on Starting Position
  - Ensure the first call in the generated sequence varies depending on the initial formation or start position of the square.

2. Parse and Validate XML Data for Calls and Formations
  - Use XML data files to load all possible calls and formations with attributes that allow pattern adjustments based on starting position.

3. Provide Flexibility for Maximum Call Count in a Sequence
  - Allow customization of the maximum number of calls in a sequence, enabling users to set specific sequence lengths.

4. Ensure Consistent Output Format for Integration and Testing
  - Output the generated sequence in a standardized list format that is easy to test and validate for integration with other components, like the sequencer.